In [26]:
# import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
import numpy as np
from sqlalchemy import create_engine
import psycopg2

# Bring in our dataframes from AWS via Postgres

In [27]:
conn = psycopg2.connect(host="realestate.cpolm7b27cpl.us-east-2.rds.amazonaws.com", port = 5432, database="realestate", user="postgres", password="password")

In [28]:
cur = conn.cursor()

In [29]:
cur.execute("""SELECT * FROM total""")

In [30]:
query_results = cur.fetchall()

In [31]:
every_home_df = pd.DataFrame(query_results)

In [32]:
every_home_df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,5505026001,Hancock Park,Ross Glenn Co Tr Ross Glenn Co Tr Ross Family ...,Yes,No,354 N Plymouth Blvd,Los Angeles,90004,354 S Plymouth Blvd,Yes,...,5,0,0,0,0,0,0,0,0,0
1,5515028023,Hancock Park,Abrahamson Scott R Co Tr Abrahamson Scott R Co...,Yes,No,143 N Plymouth Blvd,Los Angeles,90004,143 N Plymouth Blvd,No,...,2,0,0,0,0,0,0,0,0,0
2,5515022015,Hancock Park,Berenson Matthew D And Erin Trs Berenson Matth...,Yes,No,253 S Plymouth Blvd,Los Angeles,90004,253 S Plymouth Blvd,No,...,3,0,0,0,0,0,0,0,0,0
3,5513010003,Hancock Park,Gindler David I And Kiki R Trs Gindler David I...,Yes,No,164 N Las Palmas Ave,Los Angeles,90004,164 N Las Palmas Ave,No,...,6,0,0,0,0,0,0,0,0,0
4,5515012030,Hancock Park,Goldman David And Heather Trs D And H Goldman ...,Yes,No,108 S Rossmore Ave,Los Angeles,90004,11812 San Vicente Blvd Fl4,Yes,...,7,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39305,5554010042,West Hollywood,Gregory E Baldwin,No,No,1233 N Laurel Ave #110,West Hollywood,90046,1233 N Laurel Ave #110,No,...,1,0,0,0,0,0,0,YES,0,0
39306,5560023079,West Hollywood,Colin M Thomas,No,No,1145 Larrabee St #9,West Hollywood,90069,1145 Larrabee St #9,No,...,1,0,0,0,0,0,0,YES,0,0
39307,5554012051,West Hollywood,Brooke W Bowman,No,No,7921 Norton Ave,West Hollywood,90046,7921 Norton Ave,No,...,1,0,0,0,0,0,0,YES,0,0
39308,5554008098,West Hollywood,Sara Laucius,No,No,1328 Havenhurst Dr #104,West Hollywood,90046,1328 Havenhurst Dr #104,No,...,1,0,0,0,0,0,0,YES,0,0


In [33]:
Beverly_Grove_df = every_home_df[every_home_df[1] == 'Beverly Grove']

In [34]:
len(Beverly_Grove_df)

6730

# Bring in our Sales Dataframe from AWS via Postgres

In [68]:
conn = psycopg2.connect(host="realestate.cpolm7b27cpl.us-east-2.rds.amazonaws.com", port = 5432, database="realestate", user="postgres", password="password")

In [69]:
cur = conn.cursor()

In [70]:
cur.execute("""SELECT * FROM public.sales Where neighborhood = 'Beverly Grove'""")
#cur.execute("""SELECT * FROM public.sales""")

In [71]:
query_results = cur.fetchall()

In [72]:
BG_Sales_df = pd.DataFrame(query_results)

In [73]:
BG_Sales_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,5.089002e+09,Beverly Grove,Astrid,Meghrigian,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,5.086013e+09,Beverly Grove,None,6000 Sv Holdings Llc,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,5.512016e+09,Beverly Grove,Scott,Strumwasser,None,None,5971,3Rd St,5971 W 3Rd St,None,...,None,None,None,None,None,None,None,None,None,None


# Preprocess our Beverly_Grove_df

In [76]:
## Change the datatype of our first column

Beverly_Grove_df[0] = Beverly_Grove_df[0].astype(int)
Beverly_Grove_df[0].dtype

KeyError: 0

In [75]:
# rename our first column and make it our index

Beverly_Grove_df = Beverly_Grove_df.rename(columns={0:'PARCEL'})
Beverly_Grove_df.set_index('PARCEL', inplace=True)
Beverly_Grove_df.head()

KeyError: "None of ['PARCEL'] are in the columns"

In [77]:
# drop any null values

Beverly_Grove_df.dropna()

,Neighborhood,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Beverly Grove,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,Beverly Grove,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,...,1,0,0,0,0,0,0,YES,0,0
5085032022,Beverly Grove,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,...,1,0,0,0,0,0,0,YES,0,0
5510005006,Beverly Grove,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,...,5,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,Beverly Grove,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335003015,Beverly Grove,La Peer Drive llc,No,Yes,115 N La Peer Dr,Los Angeles,90048,141 E Jefferson Blvd,Yes,Los Angeles,...,0,0,0,0,0,0,0,0,0,0
4335010018,Beverly Grove,Cobra Acquisitions llc,No,Yes,123 S La Peer Dr,Los Angeles,90048,11301 W Olympic Blvd #206,Yes,Los Angeles,...,0,0,0,0,0,0,0,0,0,0
5510018012,Beverly Grove,Aron B Bick Bar Kochba Trust,Yes,No,6217 Lindenhurst Ave,Los Angeles,90048,6217 Lindenhurst Ave,No,Los Angeles,...,0,0,0,0,YES,0,YES,FORCED,0,0


In [44]:
# These are what the column names should be:

new_column_names = ['Neighborhood','Composite Name','Owned by Trust?',
                    'Owned by Business?','SITEADDRESS','SITECITY','SITEZIP',
                    'MAILADDRESS','MAIL DIFFERENT FROM SITE?','MAILCITY','MAILSTATE',
                    'MAIL OUTSIDE CA?','TITLECO1','ASSDTOTAL','ASSDLAND',
                    'ASSDSTCT','ASSDOTHR','EXEMPTCD','EXMPTAMT','PCNTIMPD',
                    'TAXAMT','DOCDATE_YEAR','DEEDTYPE','MULTIPARCEL','PRICE',
                    'LENDER1','LOANAMOUT1','LOANTYPE1','YEARBLT','EFFYRBLT',
                    'LOTSQFT','TOTALSF','GRGTYPE','BLDGMAT','TOTUNITS',
                    'QUALCLAS','BLDGCOND','NOSTORY','ROOFMAT','FOUNDATN',
                    'BEDROOMS','BATHROOMS','FAMILYRM','DININGRM','POOL',
                    'PATIO','FIREPLCE','AIRMTHOD','HEATMTHD','WALLHGT','VIEW']

In [45]:
# Map the new column names onto our dataframe

Beverly_Grove_df.columns = new_column_names
Beverly_Grove_df.head()

,Neighborhood,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Beverly Grove,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,Beverly Grove,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,...,1,0,0,0,0,0,0,YES,0,0
5085032022,Beverly Grove,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,...,1,0,0,0,0,0,0,YES,0,0
5510005006,Beverly Grove,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,...,5,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,Beverly Grove,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


# Preprocess our BG_Sales_df

In [46]:
# Change the datatype of our first column

BG_Sales_df[0] = BG_Sales_df[0].astype(int)
BG_Sales_df[0].dtype

dtype('int64')

In [47]:
# Rename our first column to 'PARCEL' and set it as our index

BG_Sales_df = BG_Sales_df.rename(columns={0:'PARCEL'})
BG_Sales_df.set_index('PARCEL', inplace=True)
BG_Sales_df.head()

,1,2,3,4,5,6,7,8,9,10,...,35,36,37,38,39,40,41,42,43,44
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Beverly Grove,Astrid,Meghrigian,None,None,None,None,None,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None
5086013015,Beverly Grove,None,6000 Sv Holdings Llc,None,None,None,None,None,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None
5512016010,Beverly Grove,Scott,Strumwasser,None,None,5971,3Rd St,5971 W 3Rd St,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None
5511028030,Beverly Grove,None,Hart Third Street Llc,None,None,8038,3Rd St,8038 W 3Rd St,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None
5511029032,Beverly Grove,None,Triple Heights Llc,None,None,8124,3Rd St,8124 W 3Rd St,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None


In [51]:
# Here are what the column names should be. # Use this when Koiyan's database is set up 

column_names = ['Neighborhood', 'Owner 1 First Name','Owner 1 Last Name',
                'Owner 2 First Name','Owner 2 Last Name','Situs House Number',
                'Situs Street Name','Situs Street Address','Situs Unit Number',
                'Situs City','Situs State','Situs Zip Code','Mailing Street Address',
                'Mail Unit Number','Mail City','Mail State','Mail Zip/Zip + 4','Tract',
                'Zoning','Living Area','Year Built','Number of Bedrooms','Number of Baths',
                'Land Use','County Land Use','LMS-Recording Date','LMS-Sale Price',
                'LMS-Document Number','Prior Sale Date','Prior Sale Price',
                'LMS-Deed Type','Lot Area','No. Residential Units','No. Commercial Units',
                'Stories No.','Property Tax','LMS-1st MTG Amount','Lms-1st MTG Type',
                'LMS-Lender','LMS-Title Company','Assessed Total Value',
                'Assessed Land Value','Assessed Improvement Value','School District 1']

In [52]:
# # Here are the temporary column names that you are importing from your own database

# column_names = ['Owner Name2', 'Owner1 First Name', 'Owner1 Last Name', 
#                 'Site Address', 'Site Address House Number', 'Site Address Unit Number', 
#                 'Site Address Street Name', 'Site Address City', 'Site Address State', 
#                 'Site Address Zip', 'Mail Address', 'Mail Address Unit Number', 'Mail Address City', 
#                 'Mail Address State', 'Mail Address Zip', 'Number of Units', 'Owner Occupied', 
#                 'Bathrooms', 'Pool', 'Bedrooms', 'Sale Date', 'Lot Area Acres', 'Lot Area SQFT', 
#                 'Building Area', 'Sales Price', 'Tax Delinquent', 'Tax Exemption Code', 'Tract', 
#                 'Use Code', 'Use Code Description', 'Census Tract', 'Year Built', 'Zoning']

In [53]:
# Set the desired column names to our dataframe

BG_Sales_df.columns = column_names
BG_Sales_df.head()

,Neighborhood,Owner 1 First Name,Owner 1 Last Name,Owner 2 First Name,Owner 2 Last Name,Situs House Number,Situs Street Name,Situs Street Address,Situs Unit Number,Situs City,...,Stories No.,Property Tax,LMS-1st MTG Amount,Lms-1st MTG Type,LMS-Lender,LMS-Title Company,Assessed Total Value,Assessed Land Value,Assessed Improvement Value,School District 1
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Beverly Grove,Astrid,Meghrigian,None,None,None,None,None,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None
5086013015,Beverly Grove,None,6000 Sv Holdings Llc,None,None,None,None,None,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None
5512016010,Beverly Grove,Scott,Strumwasser,None,None,5971,3Rd St,5971 W 3Rd St,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None
5511028030,Beverly Grove,None,Hart Third Street Llc,None,None,8038,3Rd St,8038 W 3Rd St,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None
5511029032,Beverly Grove,None,Triple Heights Llc,None,None,8124,3Rd St,8124 W 3Rd St,None,Los Angeles,...,NaN,None,None,None,None,None,None,None,None,None


In [34]:
# Add our 'Did it sell' column to our BG_Sales dataframe

BG_Sales_df['Did it sell?'] = 'yes'

# check the new column
BG_Sales_df.head()

,Owner Name2,Owner1 First Name,Owner1 Last Name,Site Address,Site Address House Number,Site Address Unit Number,Site Address Street Name,Site Address City,Site Address State,Site Address Zip,...,Sales Price,Tax Delinquent,Tax Exemption Code,Tract,Use Code,Use Code Description,Census Tract,Year Built,Zoning,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Astrid Meghrigian,Astrid,Meghrigian,None,NaN,None,None,Los Angeles,CA,90036,...,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2,yes
5086013015,6000 Sv Holdings Llc,None,6000 Sv Holdings Llc,None,NaN,None,None,Los Angeles,CA,90036,...,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2,yes
5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,None,3Rd St,Los Angeles,CA,90036,...,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1,yes
5511028030,Hart Third Street Llc,None,Hart Third Street Llc,8038 W 3Rd St,8038.0,None,3Rd St,Los Angeles,CA,90048,...,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2,yes
5511029032,Triple Heights Llc,None,Triple Heights Llc,8124 W 3Rd St,8124.0,None,3Rd St,Los Angeles,CA,90048,...,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2,yes


In [35]:
# Drop the useless column names, i.e everything except 'Did it sell?'

BG_Sales_df = BG_Sales_df[['Did it sell?']]

# check to see if that worked

BG_Sales_df.head()

,Did it sell?
PARCEL,
5089002004,yes
5086013015,yes
5512016010,yes
5511028030,yes
5511029032,yes


# Merge our Dataframes

In [36]:
Beverly_Grove_df = Beverly_Grove_df.merge(BG_Sales_df, how='left', left_index=True, right_index=True)
Beverly_Grove_df.head()

,Neighborhood,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,...,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Beverly Grove,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,...,0,1,0,0,YES,CENTRAL,CENTRAL,0,0,NaN
5085033008,Beverly Grove,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,...,0,0,0,0,0,0,YES,0,0,NaN
5085032022,Beverly Grove,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,...,0,0,0,0,0,0,YES,0,0,NaN
5510005006,Beverly Grove,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,...,0,1,0,0,YES,0,CENTRAL,0,0,NaN
5089024002,Beverly Grove,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,...,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0,NaN


In [37]:
# Fill in the NaN's in the 'Did it sell' column
Beverly_Grove_df['Did it sell?'] = Beverly_Grove_df['Did it sell?'].fillna('no')

In [38]:
# check the value counts
Beverly_Grove_df['Did it sell?'].value_counts()

no     6228
yes     502
Name: Did it sell?, dtype: int64

# Drop the columns we don't need

In [39]:
Beverly_Grove_df

,Neighborhood,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,...,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Beverly Grove,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,...,0,1,0,0,YES,CENTRAL,CENTRAL,0,0,no
5085033008,Beverly Grove,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,...,0,0,0,0,0,0,YES,0,0,no
5085032022,Beverly Grove,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,...,0,0,0,0,0,0,YES,0,0,no
5510005006,Beverly Grove,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,...,0,1,0,0,YES,0,CENTRAL,0,0,no
5089024002,Beverly Grove,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,...,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335003015,Beverly Grove,La Peer Drive llc,No,Yes,115 N La Peer Dr,Los Angeles,90048,141 E Jefferson Blvd,Yes,Los Angeles,...,0,0,0,0,0,0,0,0,0,no
4335010018,Beverly Grove,Cobra Acquisitions llc,No,Yes,123 S La Peer Dr,Los Angeles,90048,11301 W Olympic Blvd #206,Yes,Los Angeles,...,0,0,0,0,0,0,0,0,0,no
5510018012,Beverly Grove,Aron B Bick Bar Kochba Trust,Yes,No,6217 Lindenhurst Ave,Los Angeles,90048,6217 Lindenhurst Ave,No,Los Angeles,...,0,0,0,YES,0,YES,FORCED,0,0,no


In [41]:
# drop 'Neighborhood', Composite Name', 'SITECITY', 'SITEZIP', 'MAILADDRESS','WALLHGT'
Beverly_Grove_df.drop(['Neighborhood', 'Composite Name', 'SITECITY', 'SITEZIP','MAILADDRESS', 'WALLHGT', 'MAILCITY', 'MAILSTATE'],
                              axis=1, inplace=True)


In [42]:
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,No,No,1030 S Dunsmuir Ave,No,No,FIRST AMERICAN TITLE,"$1,157,385.00","$925,908.00","$231,477.00",$0.00,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,no
5085033008,No,No,1364 S Ridgeley Dr,Yes,No,0,"$57,047.00","$40,994.00","$16,053.00",$0.00,...,1,0,0,0,0,0,0,YES,0,no
5085032022,No,No,1322 Hauser Blvd,No,No,ATTORNEY ONLY,"$651,485.00","$521,193.00","$130,292.00",$0.00,...,1,0,0,0,0,0,0,YES,0,no
5510005006,Yes,No,6727 Drexel Ave,Yes,No,0,"$85,697.00","$74,539.00","$11,158.00",$0.00,...,5,0,1,0,0,YES,0,CENTRAL,0,no
5089024002,No,No,942 S Cochran Ave,No,No,0,"$62,487.00","$42,013.00","$20,474.00",$0.00,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,no


# Encode Column Values

In [43]:
# Determine the number of unique values in each column.

Beverly_Grove_df.apply(lambda x: len(x.unique()))

Owned by Trust?                 2
Owned by Business?              2
SITEADDRESS                  6716
MAIL DIFFERENT FROM SITE?       2
MAIL OUTSIDE CA?                2
TITLECO1                      119
ASSDTOTAL                    6137
ASSDLAND                     5124
ASSDSTCT                     5603
ASSDOTHR                        5
EXEMPTCD                        2
EXMPTAMT                       14
PCNTIMPD                       94
TAXAMT                       6671
DOCDATE_YEAR                   58
DEEDTYPE                       23
MULTIPARCEL                     3
PRICE                        1172
LENDER1                       504
LOANAMOUT1                   1394
LOANTYPE1                      11
YEARBLT                        98
EFFYRBLT                       99
LOTSQFT                      1757
TOTALSF                      2186
GRGTYPE                         7
BLDGMAT                         9
TOTUNITS                       34
QUALCLAS                        6
BLDGCOND      

In [44]:
# Convert binary value counts to 0's and 1's, since most of them are 'Yes' or 'No' right now.
# Step 1: 'Owned by Trust?'

Beverly_Grove_df['Owned by Trust?'].replace({"Yes": 1, "No": 0}, inplace=True)

In [45]:
# Step 2: 'Owned by Business?'
Beverly_Grove_df['Owned by Business?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 3: 'MAIL DIFFERENT FROM SITE?'
Beverly_Grove_df['MAIL DIFFERENT FROM SITE?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 4: 'MAIL OUTSIDE CA'
Beverly_Grove_df['MAIL OUTSIDE CA?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 5: 'EXEMPTCD'
Beverly_Grove_df['EXEMPTCD'].replace({"HOMEOWNERS": 1}, inplace=True)

# Step 6: 'Did it sell?'
Beverly_Grove_df['Did it sell?'].replace({"yes": 1, "no": 0}, inplace=True)

In [46]:
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,FIRST AMERICAN TITLE,"$1,157,385.00","$925,908.00","$231,477.00",$0.00,...,2,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,"$57,047.00","$40,994.00","$16,053.00",$0.00,...,1,0,0,0,0,0,0,YES,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,ATTORNEY ONLY,"$651,485.00","$521,193.00","$130,292.00",$0.00,...,1,0,0,0,0,0,0,YES,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,"$85,697.00","$74,539.00","$11,158.00",$0.00,...,5,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,"$62,487.00","$42,013.00","$20,474.00",$0.00,...,1,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


# Now encode the categorical data

In [47]:
Beverly_Grove_df.columns

Index(['Owned by Trust?', 'Owned by Business?', 'SITEADDRESS',
       'MAIL DIFFERENT FROM SITE?', 'MAIL OUTSIDE CA?', 'TITLECO1',
       'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT', 'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT',
       'PCNTIMPD', 'TAXAMT', 'DOCDATE_YEAR', 'DEEDTYPE', 'MULTIPARCEL',
       'PRICE', 'LENDER1', 'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT',
       'LOTSQFT', 'TOTALSF', 'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS',
       'BLDGCOND', 'NOSTORY', 'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS',
       'FAMILYRM', 'DININGRM', 'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD',
       'HEATMTHD', 'VIEW', 'Did it sell?'],
      dtype='object')

In [48]:
Encoded_List = [
    'TITLECO1',
    'MULTIPARCEL',
    'LENDER1',
    'LOANTYPE1',
    'GRGTYPE',
    'BLDGMAT',
    'QUALCLAS',
    'BLDGCOND',
    'POOL',
    'PATIO',
    'FIREPLCE',
    'AIRMTHOD',
    'HEATMTHD',
    'VIEW',
    'ROOFMAT',
    'FOUNDATN',
]

In [49]:
for col in Beverly_Grove_df:
    if col in Encoded_List:
        Beverly_Grove_df[col] = Beverly_Grove_df[col].transform(lambda x: x != '0')
        Beverly_Grove_df[col] = Beverly_Grove_df[col].astype(int)

In [50]:
Beverly_Grove_df.head()

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,"$1,157,385.00","$925,908.00","$231,477.00",$0.00,...,2,0,1,0,0,1,1,1,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,"$57,047.00","$40,994.00","$16,053.00",$0.00,...,1,0,0,0,0,0,0,1,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,"$651,485.00","$521,193.00","$130,292.00",$0.00,...,1,0,0,0,0,0,0,1,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,"$85,697.00","$74,539.00","$11,158.00",$0.00,...,5,0,1,0,0,1,0,1,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,"$62,487.00","$42,013.00","$20,474.00",$0.00,...,1,1,0,0,1,1,1,1,0,0


In [51]:
Beverly_Grove_df.dtypes

Owned by Trust?                int64
Owned by Business?             int64
SITEADDRESS                   object
MAIL DIFFERENT FROM SITE?      int64
MAIL OUTSIDE CA?               int64
TITLECO1                       int64
ASSDTOTAL                     object
ASSDLAND                      object
ASSDSTCT                      object
ASSDOTHR                      object
EXEMPTCD                      object
EXMPTAMT                      object
PCNTIMPD                      object
TAXAMT                       float64
DOCDATE_YEAR                  object
DEEDTYPE                      object
MULTIPARCEL                    int64
PRICE                         object
LENDER1                        int64
LOANAMOUT1                    object
LOANTYPE1                      int64
YEARBLT                       object
EFFYRBLT                      object
LOTSQFT                       object
TOTALSF                       object
GRGTYPE                        int64
BLDGMAT                        int64
T

# Convert remaining object-type columns into ints

In [52]:
Beverly_Grove_df['YEARBLT'] = Beverly_Grove_df['YEARBLT'].astype(int)
Beverly_Grove_df['EFFYRBLT'] = Beverly_Grove_df['EFFYRBLT'].astype(float)
Beverly_Grove_df['LOTSQFT'] = Beverly_Grove_df['LOTSQFT'].astype(int)
Beverly_Grove_df['TOTALSF'] = Beverly_Grove_df['TOTALSF'].astype(int)
Beverly_Grove_df['TOTUNITS'] = Beverly_Grove_df['TOTUNITS'].astype(int)
Beverly_Grove_df['NOSTORY'] = Beverly_Grove_df['NOSTORY'].astype(float)
Beverly_Grove_df['BATHROOMS'] = Beverly_Grove_df['BATHROOMS'].astype(int)
Beverly_Grove_df['BEDROOMS'] = Beverly_Grove_df['BEDROOMS'].astype(int)
Beverly_Grove_df['FAMILYRM'] = Beverly_Grove_df['FAMILYRM'].astype(int)
Beverly_Grove_df['DININGRM'] = Beverly_Grove_df['DININGRM'].astype(int)
Beverly_Grove_df['EXEMPTCD'] = Beverly_Grove_df['EXEMPTCD'].astype(int)
Beverly_Grove_df['EXMPTAMT'] = Beverly_Grove_df['EXMPTAMT'].astype(int)
Beverly_Grove_df['PCNTIMPD'] = Beverly_Grove_df['PCNTIMPD'].astype(int)

In [53]:
Beverly_Grove_df.dtypes

Owned by Trust?                int64
Owned by Business?             int64
SITEADDRESS                   object
MAIL DIFFERENT FROM SITE?      int64
MAIL OUTSIDE CA?               int64
TITLECO1                       int64
ASSDTOTAL                     object
ASSDLAND                      object
ASSDSTCT                      object
ASSDOTHR                      object
EXEMPTCD                       int64
EXMPTAMT                       int64
PCNTIMPD                       int64
TAXAMT                       float64
DOCDATE_YEAR                  object
DEEDTYPE                      object
MULTIPARCEL                    int64
PRICE                         object
LENDER1                        int64
LOANAMOUT1                    object
LOANTYPE1                      int64
YEARBLT                        int64
EFFYRBLT                     float64
LOTSQFT                        int64
TOTALSF                        int64
GRGTYPE                        int64
BLDGMAT                        int64
T

In [54]:
for column in ['EXMPTAMT', 'DOCDATE_YEAR', 'ASSDTOTAL', 'ASSDLAND', 
               'ASSDSTCT', 'ASSDOTHR', 'PRICE', 'LOANAMOUT1']:
    Beverly_Grove_df[column] = Beverly_Grove_df[column].transform(lambda x: x.replace('$','').replace(',',''))
    Beverly_Grove_df[column] = Beverly_Grove_df[column].astype(float).astype(int)

In [55]:
Beverly_Grove_df.dtypes

Owned by Trust?                int64
Owned by Business?             int64
SITEADDRESS                   object
MAIL DIFFERENT FROM SITE?      int64
MAIL OUTSIDE CA?               int64
TITLECO1                       int64
ASSDTOTAL                      int64
ASSDLAND                       int64
ASSDSTCT                       int64
ASSDOTHR                       int64
EXEMPTCD                       int64
EXMPTAMT                       int64
PCNTIMPD                       int64
TAXAMT                       float64
DOCDATE_YEAR                   int64
DEEDTYPE                      object
MULTIPARCEL                    int64
PRICE                          int64
LENDER1                        int64
LOANAMOUT1                     int64
LOANTYPE1                      int64
YEARBLT                        int64
EFFYRBLT                     float64
LOTSQFT                        int64
TOTALSF                        int64
GRGTYPE                        int64
BLDGMAT                        int64
T

# Encode our Deedtype Values

In [56]:
Deedtype_Series = Beverly_Grove_df.groupby('DEEDTYPE').count()['ASSDTOTAL'].tail(-10).index

In [57]:
Deedtype_Series = Deedtype_Series.to_list()
Deedtype_Series.append('0')

In [58]:
boolean = Beverly_Grove_df['DEEDTYPE'].transform(lambda x: x in Deedtype_Series)
Beverly_Grove_df['DEEDTYPE'] = Beverly_Grove_df['DEEDTYPE'].where(boolean, 'Other')

In [59]:
Beverly_Grove_df = pd.get_dummies(Beverly_Grove_df, columns=['DEEDTYPE'], prefix="", prefix_sep="")

In [60]:
Beverly_Grove_df.columns

Index(['Owned by Trust?', 'Owned by Business?', 'SITEADDRESS',
       'MAIL DIFFERENT FROM SITE?', 'MAIL OUTSIDE CA?', 'TITLECO1',
       'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT', 'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT',
       'PCNTIMPD', 'TAXAMT', 'DOCDATE_YEAR', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW',
       'Did it sell?', '0', 'GRANT DEED', 'IN LIEU OF FORC', 'INTERSPOUSAL',
       'JOINT TENANT', 'NAME CHANGE', 'Other', 'PARTNERSHIP', 'PERSONAL REP',
       'QUIT CLAIM', 'RE-RECORD', 'TAX DEED', 'TRUST TRANSFER', 'TRUSTEES',
       'WARRANTY'],
      dtype='object')

In [61]:
Beverly_Grove_df

,Owned by Trust?,Owned by Business?,SITEADDRESS,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,...,NAME CHANGE,Other,PARTNERSHIP,PERSONAL REP,QUIT CLAIM,RE-RECORD,TAX DEED,TRUST TRANSFER,TRUSTEES,WARRANTY
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,1030 S Dunsmuir Ave,0,0,1,1157385,925908,231477,0,...,0,0,0,0,0,0,0,0,0,0
5085033008,0,0,1364 S Ridgeley Dr,1,0,0,57047,40994,16053,0,...,0,0,0,0,0,0,0,0,0,0
5085032022,0,0,1322 Hauser Blvd,0,0,1,651485,521193,130292,0,...,0,0,0,0,0,0,0,0,0,0
5510005006,1,0,6727 Drexel Ave,1,0,0,85697,74539,11158,0,...,0,0,0,0,0,0,0,0,0,0
5089024002,0,0,942 S Cochran Ave,0,0,0,62487,42013,20474,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335003015,0,1,115 N La Peer Dr,1,0,0,1177690,1177690,0,0,...,0,0,0,0,0,0,0,0,0,0
4335010018,0,1,123 S La Peer Dr,1,0,1,1346195,1346195,0,0,...,0,0,0,0,0,0,0,0,0,0
5510018012,1,0,6217 Lindenhurst Ave,0,0,0,762455,509154,253301,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
Beverly_Grove_df.drop(columns=['0'], inplace=True)

In [63]:
Beverly_Grove_df.columns

Index(['Owned by Trust?', 'Owned by Business?', 'SITEADDRESS',
       'MAIL DIFFERENT FROM SITE?', 'MAIL OUTSIDE CA?', 'TITLECO1',
       'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT', 'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT',
       'PCNTIMPD', 'TAXAMT', 'DOCDATE_YEAR', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW',
       'Did it sell?', 'GRANT DEED', 'IN LIEU OF FORC', 'INTERSPOUSAL',
       'JOINT TENANT', 'NAME CHANGE', 'Other', 'PARTNERSHIP', 'PERSONAL REP',
       'QUIT CLAIM', 'RE-RECORD', 'TAX DEED', 'TRUST TRANSFER', 'TRUSTEES',
       'WARRANTY'],
      dtype='object')

# Export our cleaned up dataframe so we can import it into a MLENV

In [64]:
Beverly_Grove_df.to_csv('Beverly_Grove_Cleaned.csv')